In [1]:
import pandas as pd 
import numpy as np 
import os

In [22]:
def data_generator():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')
    directories = directories[:4]#temporal, indica cuantos batches se van a tomar
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        temp = temp.sort_values('charge',ascending = False).groupby('event_id').head(10)
        # al poner el head en 100 se reduciria la tabla en un 98%
    
        data = pd.concat([data, temp], axis=0)
        
    
    #data = data.drop_duplicates()##
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    train_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')
    
    data['event'] = data.index
    
    
    data = pd.merge(data, train_meta[['event_id', 'azimuth', 'zenith']], left_on=data['event'], right_on=train_meta['event_id'].astype(float))
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')


     
    data = pd.concat([
        data.groupby('event').agg({'time': lambda x: list(x)}),     ## quizas sea necesario rellenar las listas chicas con ceros o algo para
        data.groupby('event').agg({'charge': lambda x: list(x)}),   ## poder aprovechar mas datos del resto de eventos
        data.groupby('event').agg({'x': lambda x: list(x)}),
        data.groupby('event').agg({'y': lambda x: list(x)}),
        data.groupby('event').agg({'z': lambda x: list(x)}),
        data.groupby('event')['azimuth'].apply(min),
        data.groupby('event')['zenith'].apply(min)
        ],axis=1)
    
    return data[['time','charge','x','y','z','azimuth','zenith']]



In [23]:
data = data_generator()

In [24]:
data.head(4)

,time,charge,x,y,z,azimuth,zenith
event,,,,,,,
110630198.0,"[10460.0, 14117.0, 9868.0, 10172.0, 6624.0, 66...","[1.274999976158142, 1.225000023841858, 1.42499...","[237.78, 472.05, -279.53, -403.14, -101.06, -1...","[-442.42, 127.9, 23.17, 3.49, 490.22, 490.22, ...","[-315.53, 78.76, 9.0, 93.31, -349.55, -349.55,...",0.842309,1.965695
110630201.0,"[34723.0, 31691.0, 31816.0, 34426.0, 33921.0, ...","[1.274999976158142, 1.225000023841858, 1.27499...","[326.85, 338.44, 330.03, -189.98, 338.44, -77....","[-209.07, 463.72, 127.2, 257.42, 463.72, -54.3...","[-284.53, 250.41, 415.76, 314.13, 284.45, -178...",0.499148,1.755306
110630217.0,"[10487.0, 10583.0, 15843.0, 10323.0, 10342.0, ...","[1.225000023841858, 1.274999976158142, 1.82500...","[-9.13, -9.13, 210.47, -9.13, -9.13, -145.45, ...","[-481.74, -481.74, -209.77, -481.74, -481.74, ...","[465.19, 414.13, -337.07, 346.05, 363.07, 485....",5.548999,2.604799
110630219.0,"[13390.0, 10023.0, 14299.0, 9339.0, 12699.0, 1...","[1.274999976158142, 1.3250000476837158, 1.5249...","[429.76, -77.8, -224.09, -101.06, -268.9, -32....","[351.02, -54.33, 470.86, 490.22, 354.24, 62.44...","[63.61, 332.5, -450.74, -77.22, -264.4, -355.2...",0.139054,1.632285


In [25]:
import tensorflow as tf 
import tensorflow_addons as tfa
import keras

In [26]:
size = 0.25
epoch = 30

test_size = int(round((len(data)*size),0))
data.sample(frac=1)


X = data[['time','charge','x','y','z']]
y_az = data['azimuth']
y_ze = data['zenith']


num_cols = len(X.columns)
num_rows = len(X)
res = list(X.sum(axis=1).to_numpy())
res = np.reshape(res, (num_rows, num_cols, -1))
X = res


X_train = X[test_size:]
X_test = X[:test_size]

y_az_train = y_az.iloc[test_size:]
y_az_test = y_az.iloc[:test_size]

y_ze_train = y_ze.iloc[test_size:]
y_ze_test = y_ze.iloc[:test_size]


In [43]:
input = keras.Input(shape=(5, 10, ))

x = keras.layers.Flatten()(input)

xa = keras.layers.Dense(16, activation='relu')(x)
xa = keras.layers.Dense(16, activation='relu')(xa)

xz = keras.layers.Dense(16, activation='relu')(x)
xz = keras.layers.Dense(16, activation='relu')(xz)

out_az = keras.layers.Dense(1, activation='relu', name='az-out')(xa)
out_ze = keras.layers.Dense(1, activation='relu', name='ze-out')(xz)


model = keras.Model( inputs = input, outputs = [out_az, out_ze])


model.compile(
    loss = {
        'az-out': tf.keras.losses.MeanAbsoluteError(),
        'ze-out': tf.keras.losses.MeanAbsoluteError(),
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001))
    
        
    
history = model.fit(x=X_train,
            y={
                'az-out': y_az_train,
                'ze-out': y_ze_train
            },
            validation_data=(X_test,             
            {
                'az-out': y_az_test,
                'ze-out': y_ze_test
            }),
              epochs=epoch,verbose=1)
    

Epoch 1/30
18750/18750 [==============================] - 58s 3ms/step - loss: 7.9841 - az-out_loss: 5.5685 - ze-out_loss: 2.4156 - val_loss: 4.6731 - val_az-out_loss: 3.1400 - val_ze-out_loss: 1.5332
Epoch 2/30
18750/18750 [==============================] - 66s 4ms/step - loss: 4.6748 - az-out_loss: 3.1406 - ze-out_loss: 1.5342 - val_loss: 4.6731 - val_az-out_loss: 3.1400 - val_ze-out_loss: 1.5332
Epoch 3/30
18750/18750 [==============================] - 57s 3ms/step - loss: 4.6748 - az-out_loss: 3.1406 - ze-out_loss: 1.5342 - val_loss: 4.6731 - val_az-out_loss: 3.1400 - val_ze-out_loss: 1.5332
Epoch 4/30
18750/18750 [==============================] - 55s 3ms/step - loss: 4.6748 - az-out_loss: 3.1406 - ze-out_loss: 1.5342 - val_loss: 4.6731 - val_az-out_loss: 3.1400 - val_ze-out_loss: 1.5332
Epoch 5/30
18750/18750 [==============================] - 53s 3ms/step - loss: 4.6748 - az-out_loss: 3.1406 - ze-out_loss: 1.5342 - val_loss: 4.6731 - val_az-out_loss: 3.1400 - val_ze-out_loss: 1.

In [ ]:

# az-out_loss: 3.1406 - ze-out_loss: 1.5342

In [88]:
pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/sample_submission.parquet').head()

,event_id,azimuth,zenith
0,2092,1,1
1,7344,1,1
2,9482,1,1


In [ ]:
def data_test():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/test/')
    directories = directories[:2]#temporal, indica cuantos batches se van a tomar
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/test/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        temp = temp.sort_values('charge',ascending = False).groupby('event_id').head(2)
        # al poner el head en 100 se reduciria la tabla en un 98%
    
        data = pd.concat([data, temp], axis=0)
        
    
    #data = data.drop_duplicates()##
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    
    data['event_id'] = data.index
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')

     
    data = pd.concat([
        data.groupby('event').agg({'event_id': lambda x: min(x)}),
        data.groupby('event').agg({'time': lambda x: list(x)}),
        data.groupby('event').agg({'charge': lambda x: list(x)}),
        data.groupby('event').agg({'x': lambda x: list(x)}),
        data.groupby('event').agg({'y': lambda x: list(x)}),
        data.groupby('event').agg({'z': lambda x: list(x)})
        ],axis=1)
    
    
    return data[['event_id','time','charge','x','y','z']]

In [ ]:
test_data = data_test()

In [ ]:
test_data[['azimuth', 'zenith']] = model.predict(test_data[['time', 'charge', 'x', 'y', 'z']])

In [ ]:
test_data[['event_id','azimuth','zenith']].to_parquet('submission.parquet')